## So far:
pad sentence and record ending index

But cannot index properly...

So how can I do batching??? Well, pad it with . zero. Just let it learn? How the effect of 0 will not be too big

In [1]:
from data_pre import data_preprocessing
from data_pre import data_preprocessing
from WAC import WAC
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import log
import torch.optim as optim
import time
from torch.utils.data import DataLoader, TensorDataset

(voc_ix, train_data,test_data, dev_data) = data_preprocessing()


runtime: 1.5185320377349854


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import log
import torch.optim as optim
from WAC_ATT import WAC_ATT


'''
load and prepare data
'''
#(voc_ix, trainX, trainy, testX, testy, devX, devy) = data_preprocessing()
#print("finish preparing data\n")

'''
set parameters
'''
## set hyperparameters
VOCAB_SIZE = len(voc_ix) + 1
EMBEDDING_DIM = 100
n_epoch = 20
batch_size = 500
eval_per = 20000/batch_size
PATH = "../model/wac_batch.pt"

## define model
model = WAC_ATT(EMBEDDING_DIM, VOCAB_SIZE)
optimizer = optim.Adagrad(model.parameters(), lr = 1e-2)

In [5]:

## training
losses = []
accs = []
i = 0
best_dev_acc = 0

myloss = torch.nn.BCELoss(weight=None, size_average=None, reduce=None, reduction='mean')
with torch.autograd.set_detect_anomaly(True):
    start = time.time()
    for epoch in range(n_epoch):
        print("epoch " + str(epoch))

        #dataloaders
        # make sure to SHUFFLE your data
        train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
        for X,y,lens in train_loader:

            if i % eval_per == 0:
                print("time: {}".format(time.time() - start))
                acc = model.evaluate(dev_data.tensors)
                if acc > best_dev_acc:
                    best_dev_acc = acc
                    torch.save(model, PATH)
                print("accuracy on dev: " + str(acc))
                accs.append(acc)

            # Step 3. Run our forward pass.
            prob = model.forward(X, lens)

            # Step 4. Compute the loss, gradients, and update the parameters by
            #  calling optimizer.step()
            #loss_sent = - y*log(prob) - (1-y)*log(1-prob)
            loss = myloss(prob, y.float())
            #loss += loss_sent

            #import pdb; pdb.set_trace()
            loss.backward()
            optimizer.step()
            model.zero_grad()
            i +=1

        losses.append(loss.item())
        runtime = time.time() - start
    print("runtime: " + str(runtime) + "s")

model_best = torch.load(PATH)
model_best.eval()
acc_dev = model_best.evaluate(dev_data.tensors)
print("best model acc on dev: " + str(acc_dev))
acc_test = model_best.evaluate(test_data.tensors)
print("best model acc on test: " + str(acc_test))

epoch 0
time: 0.019526004791259766
accuracy on dev: 0.5103211009174312


/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([500])) that is different to the input size (torch.Size([500, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


time: 13.11504602432251
accuracy on dev: 0.5045871559633027
time: 26.345216035842896
accuracy on dev: 0.5045871559633027
time: 39.15019106864929
accuracy on dev: 0.5103211009174312


/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([349])) that is different to the input size (torch.Size([349, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


epoch 1
time: 52.1995689868927
accuracy on dev: 0.5137614678899083
time: 65.25955605506897
accuracy on dev: 0.5068807339449541


KeyboardInterrupt: 

In [ ]:
att

In [ ]:
X.shape

In [ ]:
model.forward(X,lens)

In [ ]:
# obtain one batch of training data
batch_size = 500
train_loader = DataLoader(train_data, shuffle=F, batch_size=batch_size)

dataiter = iter(train_loader)
X, y,lens = dataiter.next()

embeds = model.word_embeddings(X)
embeds.shape

In [ ]:
X.shape

In [ ]:
maxlen = X.size(1)
mask = torch.arange(maxlen)[None,:] < lens[:,None]
embeds[~mask] = float(0)

In [ ]:
X[0,:]

In [ ]:
(embeds.sum(dim = 1).t()/lens.float()).t()[0,:]/model.word_embeddings(X[0,:6]).mean(dim = 0)

In [ ]:
model.word_embeddings(X[0,:9]).shape

In [ ]:
lens

In [ ]:
maxlen/lens

In [ ]:
embeds.shape

In [ ]:
embeds[2,6,:]

In [ ]:
(embeds.mean(dim = 1).t()*(maxlen/lens.float())).t()[0,:].shape

In [ ]:
torch.matmul(maxlen/lens.float(),embeds.mean(dim = 1)).shape

In [ ]:
model.predict(X)

In [ ]:
model.evaluate(dev_data.tensors)

In [ ]:
dev_data.tensors[1].shape

In [ ]:
x = dev_data.tensors[0][0,:]
y = dev_data.tensors[1][0]

model.forward(x.unsqueeze(0),lens = torch.Tensor(1))

In [ ]:
model.forward(x.unsqueeze(0),lens = torch.tensor([1], dtype = torch.long))

In [ ]:
dtype=torch.long, device=device)